In [ ]:
"""
WhatsApp Messenger Bot using Selenium
---------------------------------------------------
IMPORTANT:
- This tool is provided for educational and personal use only.
- Ensure you have recipient consent, and be aware that bulk messaging may violate WhatsApp’s terms of service.
- Use responsibly.
"""

import sys
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [ ]:
# -------------------------------
# CONFIGURATION PARAMETERS
# -------------------------------
EXCEL_FILE = "contacts.xlsx"   # Excel file with columns 'Business Name' and 'Phone Number'
BATCH_SIZE = 10                # Number of messages per batch
BATCH_DELAY = 60               # Delay (in seconds) between batches
RETRY_LIMIT = 1                # Number of retry attempts per message

# Global log list to capture delivery details
LOG_LIST = []

In [ ]:
# -------------------------------
# Data Handling Module
# -------------------------------
def load_contacts(excel_file):
    try:
        df = pd.read_excel(excel_file)
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        sys.exit(1)
    if 'Business Name' not in df.columns or 'Phone Number' not in df.columns:
        print("Excel file must contain 'Business Name' and 'Phone Number' columns.")
        sys.exit(1)
    # Read and clean phone numbers and business names
    contacts = df['Phone Number'].astype(str).str.strip().tolist()
    business_names = df['Business Name'].tolist()
    # Ensure phone numbers have a '+' at the beginning for international format
    contacts = [phone if phone.startswith('+') else '+' + phone for phone in contacts]
    # Create personalized messages for each contact
    messages = [f"Hello {name}, thank you for your support! - Regards Apex Strategies Inc." for name in business_names]
    return contacts, messages

def display_summary(contacts):
    total = len(contacts)
    print(f"\nPre-Send Summary: {total} contacts loaded.")
    confirm = input("Do you want to proceed with sending messages? (yes/no): ")
    if confirm.lower() != 'yes':
        print("Exiting.")
        sys.exit(0)

In [ ]:
# -------------------------------
# Authentication Module (Selenium)
# -------------------------------
def authenticate_whatsapp():
    print("\nLaunching WhatsApp Web for authentication...")
    driver = webdriver.Chrome()  # Ensure chromedriver is in PATH or specify executable_path
    driver.get("https://web.whatsapp.com")
    print("Please scan the QR code with your phone to authenticate WhatsApp Web.")
    try:
        # Wait until the chat sidebar appears (up to 60 seconds)
        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.ID, "pane-side")))
    except Exception as e:
        print("Timed out waiting for WhatsApp authentication. Exiting.")
        driver.quit()
        sys.exit(1)
    print("Authentication successful!")
    # Wait for session to stabilize
    time.sleep(5)
    return driver


In [ ]:
# -------------------------------
# Messaging Module (Using Selenium)
# -------------------------------
def send_messages(contacts, messages, driver):
    total = len(contacts)
    for i in range(total):
        phone = contacts[i]
        msg = messages[i]
        success = False
        attempts = 0
        chat_url = f"https://web.whatsapp.com/send?phone={phone}&text&app_absent=0"
        while not success and attempts <= RETRY_LIMIT:
            try:
                print(f"\nNavigating to chat for {phone} (Attempt {attempts + 1})...")
                driver.get(chat_url)
                # Wait for the message input box to appear; XPath may change with WhatsApp updates.
                input_box = WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@contenteditable='true'][@data-tab='10']"))
                )
                time.sleep(3)  # Extra wait to ensure the page loads completely
                input_box.click()
                input_box.clear()
                input_box.send_keys(msg)
                input_box.send_keys(Keys.ENTER)
                print(f"Message successfully sent to {phone}")
                LOG_LIST.append({
                    "Phone Number": phone,
                    "Status": "Sent",
                    "Message": msg,
                    "Attempts": attempts + 1
                })
                success = True
                time.sleep(3)  # Brief pause before processing the next contact
            except Exception as e:
                attempts += 1
                print(f"Error sending message to {phone}: {e}")
                if attempts > RETRY_LIMIT:
                    LOG_LIST.append({
                        "Phone Number": phone,
                        "Status": "Failed",
                        "Message": msg,
                        "Error": str(e),
                        "Attempts": attempts
                    })
                else:
                    print("Retrying in 5 seconds...")
                    time.sleep(5)
        # Batch processing: pause after processing each batch
        if (i + 1) % BATCH_SIZE == 0:
            print(f"\nBatch of {BATCH_SIZE} messages processed. Waiting for {BATCH_DELAY} seconds before next batch...")
            time.sleep(BATCH_DELAY)


In [ ]:
# -------------------------------
# Logging & Reporting Module
# -------------------------------
def save_log(log_list, filename="delivery_log.xlsx"):
    try:
        df_log = pd.DataFrame(log_list)
        df_log.to_excel(filename, index=False)
        print(f"\nDelivery log saved to '{filename}'.")
    except Exception as e:
        print(f"Error saving log: {e}")


In [ ]:
# -------------------------------
# Main Workflow
# -------------------------------
def main():
    print("Starting WhatsApp Messenger Bot...\n")
    # 1. Load contacts and personalized messages from Excel
    contacts, messages = load_contacts(EXCEL_FILE)
    display_summary(contacts)
    
    # 2. Authenticate on WhatsApp Web using Selenium
    driver = authenticate_whatsapp()
    
    # 3. Send messages using Selenium (persistent session)
    send_messages(contacts, messages, driver)
    
    # 4. Save the delivery log to an Excel file
    save_log(LOG_LIST)
    
    print("\nAll messages have been processed.")
    driver.quit()


In [ ]:
# -------------------------------
# Entry Point
# -------------------------------
if __name__ == "__main__":
    main()

